In [18]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_columns', 999)

#### Connect to SQLite database

In [5]:
db = sqlite3.connect('data/NPPES_Data_Dissemination_February_2021/npidata2.sqlite')

#### SQL query for npi data

In [9]:
query = """
        SELECT * 
        FROM npidata
        LIMIT 5
 """

#### Assign query to variable

In [10]:
npi_sqlite = pd.read_sql(query, db) 

#### View npi data variable

In [17]:
npi_sqlite


,NPI,Entity_Type_Code,Provider_Organization_Name,Provider_Last_Name,Provider_First_Name,First_Line_Address,Second_Line_Address,Address_City,Address_Postal_Codes,Provider_Credential_Text,Gender,Healthcare_Provider_Taxonomy_Code
0,1841293891,1.0,None,GIBBS,ELMER,49 CLEVELAND ST 310,None,CROSSVILLE,38555,M.D.,M,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,105 W STONE DR,STE 1F,KINGSPORT,37660,MD,F,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,250 25TH AVE N,STE 412,NASHVILLE,37203,CRNP,F,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,345 23RD AVE N,SUITE 209,NASHVILLE,37203,M.D.,F,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,219 CHURCH ST,None,DICKSON,37055,MD,F,207Q00000X


#### Run query to join npi data, CBSA data and Taxonomy classification with filter on General Acute Care Hospitals, entity type 2 and CBSA

In [15]:
query1 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.to_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 2.0 AND Classification = 'General Acute Care Hospital'AND CBSA = 34980  
 """

#### Assign variable to above query 

In [16]:
npi_sqlite1 = pd.read_sql(query1, db)

#### View query

In [19]:
npi_sqlite1

,NPI,Entity_Type_Code,Provider_Organization_Name,Provider_Last_Name,Provider_First_Name,First_Line_Address,Second_Line_Address,Address_City,Address_Postal_Codes,Provider_Credential_Text,Gender,Healthcare_Provider_Taxonomy_Code,Code,Grouping,Classification,Specialization,Definition,Effective Date,Deactivation Date,Last Modified Date,Notes,Display Name,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,ZIP,CBSA,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1023055126,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,2300 PATTERSON ST,None,NASHVILLE,37203,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1003013160,1023055126,13,23,31.870,36.145,37203,34980,1.000000,1.000000,1.0,1.000000
1,1093769606,2.0,HTI MEMORIAL HOSPITAL CORPORATION,None,None,3441 DICKERSON PIKE,None,NASHVILLE,37207,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1003863580,1093769606,46,47,13.681,44.739,37207,34980,1.000000,1.000000,1.0,1.000000
2,1104137215,2.0,RIVERVIEW MEDICAL CENTER LLC,None,None,158 HOSPITAL DR,None,CARTHAGE,37030,None,None,282NC0060X,282NC0060X,Hospitals,General Acute Care Hospital,Critical Access,Definition to come.,10/1/2003,None,None,[7/1/2003: new],Critical Access Hospital,1013118702,1104137215,14,14,43.357,55.950,37030,34980,0.999712,1.000000,1.0,0.999756
3,1124065255,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,391 WALLACE RD,None,NASHVILLE,37211,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1003863580,1124065255,54,55,0.000,0.000,37211,34980,1.000000,1.000000,1.0,1.000000
4,1164469227,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,5655 FRIST BLVD,None,HERMITAGE,37076,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1104295070,1164469227,18,19,6.316,25.131,37076,34980,1.000000,1.000000,1.0,1.000000
5,1164590386,2.0,SAINT THOMAS RUTHERFORD HOSPITAL,None,None,1700 MEDICAL CENTER PKWY,None,MURFREESBORO,37129,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1003028770,1164590386,63,73,83.767,87.828,37129,34980,1.000000,1.000000,1.0,1.000000
6,1184987505,2.0,"TRUSTPOINT HOSPITAL, LLC",None,None,1009 N. THOMPSON LN,None,MURFREESBORO,37128,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1013109511,1184987505,23,35,16.800,39.899,37128,34980,1.000000,1.000000,1.0,1.000000
7,1245541358,2.0,SMITH COUNTY MEMORIAL HOSPITAL LLC,None,None,158 HOSPITAL DR,None,CARTHAGE,37030,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1033182704,1245541358,12,13,22.538,66.993,37030,34980,0.999712,1.000000,1.0,0.999756
8,1265445506,2.0,WILLIAMSON COUNTY HOSPITAL DISTRICT,None,None,4321 CAROTHERS PARKWAY,None,FRANKLIN,37067,None,None,282N00000X,282N00000X,Hospitals,General Acute Care Hospital,None,An acute general hospital is an institution wh...,4/1/2002,None,None,None,General Acute Care Hospital,1003019902,1265445506,14,15,44.600,32.980,37067,34980,1.000000,1.000000,1.0,1.000000
9,1265487193,2.0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,313 N MAIN ST,None,ASHLAND CITY,37015,None,None,282NC0060X,282NC0060X,Hospitals,General Acute Care Hospital,Critical Access,Definition to come.,10/1/2003,None,None,[7/1/2003: new],Critic

#### Run query to join npi data, CBSA data and Taxonomy classification with filter on entity type 1 and CBSA

In [20]:
query2 = """
        SELECT *
        FROM npidata AS n
        INNER JOIN taxonomy AS t
        on n.Healthcare_Provider_Taxonomy_Code = t.Code
        INNER JOIN hops AS h 
        on n.NPI = h.from_npi
        INNER JOIN CBSA as c
        on n.Address_Postal_Codes = c.ZIP
        WHERE Entity_Type_Code = 1.0 AND CBSA = 34980  
 """

#### Assign variable to above query 

In [21]:
npi_sqlite2 = pd.read_sql(query2, db)

#### View query above

In [ ]:
npi_sqlite2

Questions to consider with the data:
- How do we combine and filter datasets to have npi_from to be entity type 1's only and to_npi to be entity type 2's only
- Once we have the completed dataset, Are there any filters we want to put on taxonomy classification types (i.e. drama therapists, veterinarian, bus, train etc.)
- What hospitals are we considering in this analysis?